# 아이템 기반 추천 시스템 구현 

- 소비자에게 하나의 상품과 유사한 특성을 가진 다른 상품들을 추천

In [309]:
#라이브러리 실행
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [310]:
#데이터 불러오기
data=pd.read_excel('아이템기반최종데이터.xlsx')
data.head()

,Unnamed: 0,상품명,색깔,카테고리,RGB,패턴재질,모양,계절,평균평점,스타일,종합점수
0,0,[DAYONE LABEL]렌프 자켓원피스(벨트set),블랙,미니원피스,"0, 0, 0",무지,단추,겨울,0.0,모던시크,0.0
1,1,[DAYONE LABEL]로티 숏팬츠,연청,반바지,"190, 210, 237",무지,포켓,봄 여름,0.0,심플베이직,0.0
2,2,[DAYONE LABEL]슬로 보트넥티,옐로우,티셔츠,"245, 230, 160",무지,보트넥,봄 가을,0.0,청순,0.0
3,3,[DAYONE LABEL]워프 스퀘어롱원피스,피치핑크,롱원피스,"220, 174, 166",데님,크롭,여름,0.0,청순,0.0
4,4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),블루,스키니,"85, 100, 113",무지,슬림,겨울,0.0,캐주얼,0.0


### 여러 벡터 조건을 충족시키기 위해  사전에 여러 칼럼들을 하나로 합치기

- 상품이름 : 상품명 + 색깔
- 종합컬럼 : 스타일 + 색깔 + 계절 + 카테고리 + 모양 + 패턴재질

In [311]:
data['상품이름']=data['상품명']+'-'+data['색깔']
data.head()

,Unnamed: 0,상품명,색깔,카테고리,RGB,패턴재질,모양,계절,평균평점,스타일,종합점수,상품이름
0,0,[DAYONE LABEL]렌프 자켓원피스(벨트set),블랙,미니원피스,"0, 0, 0",무지,단추,겨울,0.0,모던시크,0.0,[DAYONE LABEL]렌프 자켓원피스(벨트set)-블랙
1,1,[DAYONE LABEL]로티 숏팬츠,연청,반바지,"190, 210, 237",무지,포켓,봄 여름,0.0,심플베이직,0.0,[DAYONE LABEL]로티 숏팬츠-연청
2,2,[DAYONE LABEL]슬로 보트넥티,옐로우,티셔츠,"245, 230, 160",무지,보트넥,봄 가을,0.0,청순,0.0,[DAYONE LABEL]슬로 보트넥티-옐로우
3,3,[DAYONE LABEL]워프 스퀘어롱원피스,피치핑크,롱원피스,"220, 174, 166",데님,크롭,여름,0.0,청순,0.0,[DAYONE LABEL]워프 스퀘어롱원피스-피치핑크
4,4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),블루,스키니,"85, 100, 113",무지,슬림,겨울,0.0,캐주얼,0.0,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩)-블루


In [312]:
data['종합컬럼']=data['스타일']+'-'+data['색깔']+'-'+data['계절']+'-'+data['카테고리']+'-'+data['모양']+'-'+data['패턴재질']
data[50:66]

,Unnamed: 0,상품명,색깔,카테고리,RGB,패턴재질,모양,계절,평균평점,스타일,종합점수,상품이름,종합컬럼
50,50,[La belle rose]론다 플리츠 미니스커트,베이지,미니스커트,"221, 219, 200",무지,플리츠,봄 가을,0.0,러블리,0.00,[La belle rose]론다 플리츠 미니스커트-베이지,러블리-베이지-봄 가을 -미니스커트-플리츠-무지
51,51,[La belle rose]론다 플리츠 미니스커트,블랙,미니스커트,"0, 0, 0",무지,플리츠,봄 가을,0.0,러블리,0.00,[La belle rose]론다 플리츠 미니스커트-블랙,러블리-블랙-봄 가을 -미니스커트-플리츠-무지
52,52,[La belle rose]론다 플리츠 미니스커트,블랙,미니스커트,"0, 0, 0",무지,플리츠,봄 가을,0.0,러블리,0.00,[La belle rose]론다 플리츠 미니스커트-블랙,러블리-블랙 -봄 가을 -미니스커트-플리츠-무지
53,53,[La belle rose]론드 스퀘어 슬리브리스탑,블랙,민소매/나시,"0, 0, 0",무지,스퀘어넥,여름,0.0,러블리,0.00,[La belle rose]론드 스퀘어 슬리브리스탑-블랙,러블리-블랙-여름 -민소매/나시-스퀘어넥-무지
54,54,[La belle rose]론드 스퀘어 슬리브리스탑,아이보리,민소매/나시,"254, 255, 239",무지,스퀘어넥,여름,0.0,러블리,0.00,[La belle rose]론드 스퀘어 슬리브리스탑-아이보리,러블리-아이보리-여름 -민소매/나시-스퀘어넥-무지
55,55,[La belle rose]루벨 쟈가드 레이스가디건,블랙,가디건,"0, 0, 0",레이스,슬림,여름,0.0,러블리,-0.53,[La belle rose]루벨 쟈가드 레이스가디건-블랙,러블리-블랙-여름 -가디건-슬림-레이스
56,56,[La belle rose]루벨 쟈가드 레이스가디건,아이보리,가디건,"254, 255, 239",레이스,슬림,여름,0.0,러블리,-0.53,[La belle rose]루벨 쟈가드 레이스가디건-아이보리,러블리-아이보리-여름 -가디건-슬림-레이스
57,57,[La belle rose]룩스 쟈가드 랩티셔츠,블랙,티셔츠,"0, 0, 0",무지,랩,여름,0.0,러블리,-0.07,[La belle rose]룩스 쟈가드 랩티셔츠-블랙,러블리-블랙-여름 -티셔츠-랩-무지
58,58,[La belle rose]룩스 쟈가드 랩티셔츠,화이트,티셔츠,"255, 255, 255",무지,랩,여름,0.0,러블리,-0.07,[La belle rose]룩스 쟈가드 랩티셔츠-화이트,러블리-화이트-여름 -티셔츠-랩-무지
59,59,[La belle rose]리놀 스모크 나시,베이지,민소매/나시,"238, 218, 175",헤링본,크롭,여름,0.0,러블리,0.00,[La belle rose]리놀 스모크 나시-베이지,러블리-베이지-여름 -민소매/나시-크롭-헤링본


In [313]:
# 합치고 나서 데이터 컬럼 확인
data.columns

Index(['Unnamed: 0', '상품명', '색깔', '카테고리', 'RGB', '패턴재질', '모양', '계절', '평균평점',
       '스타일', '종합점수', '상품이름', '종합컬럼'],
      dtype='object')

In [314]:
# 전처리를 통해 합친 데이터, 종합점수만 추출
data=data[['상품이름','종합컬럼','종합점수']]
data.head()

,상품이름,종합컬럼,종합점수
0,[DAYONE LABEL]렌프 자켓원피스(벨트set)-블랙,모던시크-블랙-겨울 -미니원피스-단추-무지,0.0
1,[DAYONE LABEL]로티 숏팬츠-연청,심플베이직-연청-봄 여름 -반바지-포켓-무지,0.0
2,[DAYONE LABEL]슬로 보트넥티-옐로우,청순-옐로우-봄 가을 -티셔츠-보트넥-무지,0.0
3,[DAYONE LABEL]워프 스퀘어롱원피스-피치핑크,청순-피치핑크-여름 -롱원피스-크롭-데님,0.0
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩)-블루,캐주얼-블루-겨울 -스키니 -슬림-무지,0.0


### 추천 시스템을 위해 CountVectorizer 실행
 - 문자열을 숫자로 바꾸어 벡터화시킴

In [315]:
count_vector = CountVectorizer(ngram_range=(1,1)) #단어의 묶음

In [316]:
c_vector=count_vector.fit_transform(data['종합컬럼']) # 종합컬럼에 대해 벡터 수행후 적용

In [317]:
#코사인 유사도를 구한 벡터를 미리 저장
genre_c_sim=cosine_similarity(c_vector,c_vector).argsort()[:,::-1] 
# 코사인 유사도 적용 이후 argsort로 유사도가 가장 높은 인덱스를 가장 위쪽으로 정렬

In [318]:
genre_c_sim.shape

(3503, 3503)

### 추천 시스템 구현 함수 정의하고 적용

In [319]:
def get_recommend_item_list(df,상품이름,top=30):
    #특정 영화와 비슷한 영화를추천해야 하기에 '특정 영화' 정보를 추출
    target_item_index=df[df['상품이름']==상품이름].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보 추출
    sim_index=genre_c_sim[target_item_index,:top].reshape(-1)
  
    #본인 제외
    sim_index=sim_index[sim_index != target_item_index]

    #data frame로 만들고 vote_count로 정렬한뒤 return
    result=df.iloc[sim_index].sort_values('종합점수',ascending=False)[:10]
    
    #결과를 추출하기에 앞서 합쳤던 데이터들을 원래대로 분리시킴
    result['상품명']=result.상품이름.str.split('-').str[0]
    result['색깔']=result.상품이름.str.split('-').str[1]
    
    result['스타일']=result.종합컬럼.str.split('-').str[0]
    result['계절']=result.종합컬럼.str.split('-').str[2]
    result['카테고리']=result.종합컬럼.str.split('-').str[3]
    result['모양']=result.종합컬럼.str.split('-').str[4]
    result['패턴재질']=result.종합컬럼.str.split('-').str[5]
    
    # 필요 칼럼만 추가하고, 중복되서 나오게 되는 데이터는 제거
    results=result[['상품명','색깔','스타일','계절','카테고리','모양','패턴재질']]
    results=results.drop_duplicates()
    
    return results

In [320]:
#결과 확인
get_recommend_item_list(data,'[La belle rose]론다 플리츠 미니스커트-베이지')

,상품명,색깔,스타일,계절,카테고리,모양,패턴재질
267,블라디 플리츠스커트,베이지,러블리,가을,미니스커트,플리츠,무지
133,달다 셔링스커트,베이지,러블리,가을,미니스커트,셔링,무지
210,머핀 니트원피스(벨트set),베이지,러블리,가을,미니원피스,브이넥,무지
1768,에밀리 플리츠스커트,그레이,유니크,가을,미니스커트,플리츠,무지
198,리브 언발 미니스커트,아이보리,러블리,가을,미니스커트,언발,무지
1769,에밀리 플리츠스커트,블랙,유니크,가을,미니스커트,플리츠,무지
132,달다 셔링스커트,레드브라운,러블리,가을,미니스커트,셔링,무지
1704,다꼬아 플리츠스커트,베이지,유니크,겨울,미니스커트,플리츠,무지
2026,모너 언발스커트,베이지,청순,봄 가을,미니스커트,언발,무지
3384,크루 랩스커트,아이보리,러블리,봄 여름 가을,미니스커트,랩,무지


In [266]:
get_recommendations('[La belle rose]론다 플리츠 미니스커트-베이지)

,상품이름,카테고리,패턴재질,모양,계절,평균평점,스타일,종합점수
0,[La belle rose]론다 플리츠 미니스커트-블랙,미니스커트,무지,플리츠,봄 가을,0.000000,러블리,0.0
1,[La belle rose]론다 플리츠 미니스커트-블랙,미니스커트,무지,플리츠,봄 가을,0.000000,러블리,0.0
2,민니 핀턱 주름스커트-블랙,미니스커트,무지,핀턱,봄 가을,4.607383,러블리,-0.1
3,민니 핀턱 주름스커트-아이보리,미니스커트,무지,핀턱,봄 가을,4.607383,러블리,-0.1
4,민니 핀턱 주름스커트-차콜,미니스커트,무지,핀턱,봄 가을,4.607383,러블리,-0.1
5,블라디 플리츠스커트-그레이,미니스커트,무지,플리츠,가을,4.785714,러블리,0.0
6,블라디 플리츠스커트-베이지,미니스커트,무지,플리츠,가을,4.785714,러블리,0.0
7,아세르 플리츠 미니스커트-라이트퍼플,미니스커트,무지,플리츠,봄 여름 가을,4.555556,러블리,0.0
8,아세르 플리츠 미니스커트-블랙,미니스커트,무지,플리츠,봄 여름 가을,4.555556,러블리,0.0
9,아세르 플리츠 미니스커트-크림,미니스커트,무지,플리츠,봄 여름 가을,4.555556,러블리,0.0


In [90]:
# cat_train = pd.get_dummies(data = train, columns = ['공급유형'], prefix = '공급유형',)

In [284]:
data=pd.read_excel('아이템기반최종데이터.xlsx')
data.head()

,Unnamed: 0,상품명,색깔,카테고리,RGB,패턴재질,모양,계절,평균평점,스타일,종합점수
0,0,[DAYONE LABEL]렌프 자켓원피스(벨트set),블랙,미니원피스,"0, 0, 0",무지,단추,겨울,0.0,모던시크,0.0
1,1,[DAYONE LABEL]로티 숏팬츠,연청,반바지,"190, 210, 237",무지,포켓,봄 여름,0.0,심플베이직,0.0
2,2,[DAYONE LABEL]슬로 보트넥티,옐로우,티셔츠,"245, 230, 160",무지,보트넥,봄 가을,0.0,청순,0.0
3,3,[DAYONE LABEL]워프 스퀘어롱원피스,피치핑크,롱원피스,"220, 174, 166",데님,크롭,여름,0.0,청순,0.0
4,4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),블루,스키니,"85, 100, 113",무지,슬림,겨울,0.0,캐주얼,0.0


In [285]:
data['상품이름']=data['상품명']+'-'+data['색깔']
data.head()

,Unnamed: 0,상품명,색깔,카테고리,RGB,패턴재질,모양,계절,평균평점,스타일,종합점수,상품이름
0,0,[DAYONE LABEL]렌프 자켓원피스(벨트set),블랙,미니원피스,"0, 0, 0",무지,단추,겨울,0.0,모던시크,0.0,[DAYONE LABEL]렌프 자켓원피스(벨트set)-블랙
1,1,[DAYONE LABEL]로티 숏팬츠,연청,반바지,"190, 210, 237",무지,포켓,봄 여름,0.0,심플베이직,0.0,[DAYONE LABEL]로티 숏팬츠-연청
2,2,[DAYONE LABEL]슬로 보트넥티,옐로우,티셔츠,"245, 230, 160",무지,보트넥,봄 가을,0.0,청순,0.0,[DAYONE LABEL]슬로 보트넥티-옐로우
3,3,[DAYONE LABEL]워프 스퀘어롱원피스,피치핑크,롱원피스,"220, 174, 166",데님,크롭,여름,0.0,청순,0.0,[DAYONE LABEL]워프 스퀘어롱원피스-피치핑크
4,4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),블루,스키니,"85, 100, 113",무지,슬림,겨울,0.0,캐주얼,0.0,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩)-블루


In [286]:
data=data[['상품이름','스타일','계절','카테고리','모양','패턴재질']]

In [287]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [288]:
sim=cosine_similarity(data[['스타일','계절','카테고리','모양','패턴재질']])
print(sim.shape)

ValueError: could not convert string to float: '모던시크'

In [290]:
data = pd.get_dummies(data = data, columns = ['스타일'], prefix = '스타일')
data = pd.get_dummies(data = data, columns = ['계절'], prefix = '계절')
data = pd.get_dummies(data = data, columns = ['카테고리'], prefix = '카테고리')
data = pd.get_dummies(data = data, columns = ['모양'], prefix = '모양')
data = pd.get_dummies(data = data, columns = ['패턴재질'], prefix = '패턴재질')
data.head()

,상품이름,스타일_데일리,스타일_러블리,스타일_로맨틱,스타일_모던,스타일_모던시크,스타일_미니멀,스타일_볼드,스타일_섹시,스타일_스포티,...,패턴재질_퀼팅,패턴재질_크랙,패턴재질_털,패턴재질_트위드,패턴재질_퍼,패턴재질_펀칭,패턴재질_프린팅,패턴재질_플라워,패턴재질_하찌,패턴재질_헤링본
0,[DAYONE LABEL]렌프 자켓원피스(벨트set)-블랙,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,[DAYONE LABEL]로티 숏팬츠-연청,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,[DAYONE LABEL]슬로 보트넥티-옐로우,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,[DAYONE LABEL]워프 스퀘어롱원피스-피치핑크,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩)-블루,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [291]:
data=pd.read_excel('아이템기반최종데이터.xlsx')
data.head()


,Unnamed: 0,상품명,색깔,카테고리,RGB,패턴재질,모양,계절,평균평점,스타일,종합점수
0,0,[DAYONE LABEL]렌프 자켓원피스(벨트set),블랙,미니원피스,"0, 0, 0",무지,단추,겨울,0.0,모던시크,0.0
1,1,[DAYONE LABEL]로티 숏팬츠,연청,반바지,"190, 210, 237",무지,포켓,봄 여름,0.0,심플베이직,0.0
2,2,[DAYONE LABEL]슬로 보트넥티,옐로우,티셔츠,"245, 230, 160",무지,보트넥,봄 가을,0.0,청순,0.0
3,3,[DAYONE LABEL]워프 스퀘어롱원피스,피치핑크,롱원피스,"220, 174, 166",데님,크롭,여름,0.0,청순,0.0
4,4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),블루,스키니,"85, 100, 113",무지,슬림,겨울,0.0,캐주얼,0.0


In [292]:
data=data[['상품명','색깔','스타일','계절','카테고리','모양','패턴재질','종합점수']]

In [293]:
data.head()

,상품명,색깔,스타일,계절,카테고리,모양,패턴재질,종합점수
0,[DAYONE LABEL]렌프 자켓원피스(벨트set),블랙,모던시크,겨울,미니원피스,단추,무지,0.0
1,[DAYONE LABEL]로티 숏팬츠,연청,심플베이직,봄 여름,반바지,포켓,무지,0.0
2,[DAYONE LABEL]슬로 보트넥티,옐로우,청순,봄 가을,티셔츠,보트넥,무지,0.0
3,[DAYONE LABEL]워프 스퀘어롱원피스,피치핑크,청순,여름,롱원피스,크롭,데님,0.0
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),블루,캐주얼,겨울,스키니,슬림,무지,0.0


In [294]:
data=pd.read_excel('아이템기반최종데이터.xlsx')

In [295]:
data['상품이름']=data['상품명']+'-'+data['색깔']
data.head()

,Unnamed: 0,상품명,색깔,카테고리,RGB,패턴재질,모양,계절,평균평점,스타일,종합점수,상품이름
0,0,[DAYONE LABEL]렌프 자켓원피스(벨트set),블랙,미니원피스,"0, 0, 0",무지,단추,겨울,0.0,모던시크,0.0,[DAYONE LABEL]렌프 자켓원피스(벨트set)-블랙
1,1,[DAYONE LABEL]로티 숏팬츠,연청,반바지,"190, 210, 237",무지,포켓,봄 여름,0.0,심플베이직,0.0,[DAYONE LABEL]로티 숏팬츠-연청
2,2,[DAYONE LABEL]슬로 보트넥티,옐로우,티셔츠,"245, 230, 160",무지,보트넥,봄 가을,0.0,청순,0.0,[DAYONE LABEL]슬로 보트넥티-옐로우
3,3,[DAYONE LABEL]워프 스퀘어롱원피스,피치핑크,롱원피스,"220, 174, 166",데님,크롭,여름,0.0,청순,0.0,[DAYONE LABEL]워프 스퀘어롱원피스-피치핑크
4,4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),블루,스키니,"85, 100, 113",무지,슬림,겨울,0.0,캐주얼,0.0,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩)-블루


In [296]:
data=data[['상품이름','스타일','계절','카테고리','모양','패턴재질','평균평점','종합점수']]

In [297]:
data.head()

,상품이름,스타일,계절,카테고리,모양,패턴재질,평균평점,종합점수
0,[DAYONE LABEL]렌프 자켓원피스(벨트set)-블랙,모던시크,겨울,미니원피스,단추,무지,0.0,0.0
1,[DAYONE LABEL]로티 숏팬츠-연청,심플베이직,봄 여름,반바지,포켓,무지,0.0,0.0
2,[DAYONE LABEL]슬로 보트넥티-옐로우,청순,봄 가을,티셔츠,보트넥,무지,0.0,0.0
3,[DAYONE LABEL]워프 스퀘어롱원피스-피치핑크,청순,여름,롱원피스,크롭,데님,0.0,0.0
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩)-블루,캐주얼,겨울,스키니,슬림,무지,0.0,0.0


In [298]:
data = pd.get_dummies(data = data, columns = ['스타일'], prefix = '스타일')
data = pd.get_dummies(data = data, columns = ['계절'], prefix = '계절')
data = pd.get_dummies(data = data, columns = ['카테고리'], prefix = '카테고리')
data = pd.get_dummies(data = data, columns = ['모양'], prefix = '모양')
data = pd.get_dummies(data = data, columns = ['패턴재질'], prefix = '패턴재질')
data.head()

,상품이름,평균평점,종합점수,스타일_데일리,스타일_러블리,스타일_로맨틱,스타일_모던,스타일_모던시크,스타일_미니멀,스타일_볼드,...,패턴재질_퀼팅,패턴재질_크랙,패턴재질_털,패턴재질_트위드,패턴재질_퍼,패턴재질_펀칭,패턴재질_프린팅,패턴재질_플라워,패턴재질_하찌,패턴재질_헤링본
0,[DAYONE LABEL]렌프 자켓원피스(벨트set)-블랙,0.0,0.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,[DAYONE LABEL]로티 숏팬츠-연청,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,[DAYONE LABEL]슬로 보트넥티-옐로우,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,[DAYONE LABEL]워프 스퀘어롱원피스-피치핑크,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩)-블루,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [306]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
     return dot(A, B)/(norm(A)*norm(B))

In [308]:
data.set_index(['상품이름'],inplace=True)

KeyError: "None of ['상품이름'] are in the columns"

In [304]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(data, data)

In [218]:
cosine_sim

array([[ 5.        ,  1.        ,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 1.        ,  5.        ,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [ 1.        ,  1.        ,  5.        , ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [ 0.        ,  1.        ,  0.        , ..., 28.21487603,
        17.16477273, 22.64545454],
       [ 0.        ,  0.        ,  1.        , ..., 17.16477273,
        17.69140625, 17.74375   ],
       [ 1.        ,  1.        ,  1.        , ..., 22.64545454,
        17.74375   , 27.09      ]])

In [212]:
indices =pd.Series(range(3503),index=data.index)
print(indices.head())

상품이름
[DAYONE LABEL]렌프 자켓원피스(벨트set)-블랙      0
[DAYONE LABEL]로티 숏팬츠-연청               1
[DAYONE LABEL]슬로 보트넥티-옐로우             2
[DAYONE LABEL]워프 스퀘어롱원피스-피치핑크         3
[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩)-블루    4
dtype: int64


In [224]:
indices1=pd.Series(data.index)
print(indices1.head())

0      [DAYONE LABEL]렌프 자켓원피스(벨트set)-블랙
1               [DAYONE LABEL]로티 숏팬츠-연청
2             [DAYONE LABEL]슬로 보트넥티-옐로우
3         [DAYONE LABEL]워프 스퀘어롱원피스-피치핑크
4    [DAYONE LABEL]케이즈 하이스키니진 (히든밴딩)-블루
Name: 상품이름, dtype: object


In [216]:
idx = indices['[DAYONE LABEL]렌프 자켓원피스(벨트set)-블랙']

In [217]:
sim_scores = list(enumerate(cosine_sim[idx]))

In [219]:
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

In [220]:
sim_scores = sim_scores[1:11]

In [221]:
movie_indices = [i[0] for i in sim_scores]

In [222]:
movie_indices

[1712, 2614, 86, 154, 155, 440, 441, 454, 455, 456]

In [225]:
indices1[movie_indices]

1712                           레오 더블버튼스커트-차콜 
2614         [DAYONE LABEL]돌체 자켓원피스(벨트set)-블랙
86      [La belle rose]엘리제 스퀘어넥 원피스(벨트set)-블랙
154                         러바드 플레어 니트원피스-브라운
155                          러바드 플레어 니트원피스-블랙
440        [La belle rose]돌체 자켓원피스(벨트set)-베이지
441         [La belle rose]돌체 자켓원피스(벨트set)-블랙
454            [La belle rose]원앤 슬림핏 자켓원피스-블랙
455           [La belle rose]원앤 슬림핏 자켓원피스-블랙 
456            [La belle rose]원앤 슬림핏 자켓원피스-차콜
Name: 상품이름, dtype: object

In [234]:
df=pd.read_excel('아이템기반최종데이터.xlsx')
df['상품이름']=df['상품명']+'-'+df['색깔']
df=df[['상품이름','카테고리','패턴재질','모양','계절','평균평점','스타일','종합점수']]
df.head()

,상품이름,카테고리,패턴재질,모양,계절,평균평점,스타일,종합점수
0,[DAYONE LABEL]렌프 자켓원피스(벨트set)-블랙,미니원피스,무지,단추,겨울,0.0,모던시크,0.0
1,[DAYONE LABEL]로티 숏팬츠-연청,반바지,무지,포켓,봄 여름,0.0,심플베이직,0.0
2,[DAYONE LABEL]슬로 보트넥티-옐로우,티셔츠,무지,보트넥,봄 가을,0.0,청순,0.0
3,[DAYONE LABEL]워프 스퀘어롱원피스-피치핑크,롱원피스,데님,크롭,여름,0.0,청순,0.0
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩)-블루,스키니,무지,슬림,겨울,0.0,캐주얼,0.0


In [269]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]
    
    result=pd.DataFrame(indices1[movie_indices])
    laster=pd.merge(result,df,how='left' )
    
    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return laster

In [270]:
result

,상품이름,카테고리,패턴재질,모양,계절,평균평점,스타일,종합점수
0,레오 더블버튼스커트-차콜,미니원피스,무지,단추,겨울,4.875000,유니크,0.00
1,[DAYONE LABEL]돌체 자켓원피스(벨트set)-블랙,미니원피스,무지,단추,겨울,0.000000,오피스,0.00
2,[La belle rose]엘리제 스퀘어넥 원피스(벨트set)-블랙,미니원피스,무지,스퀘어넥,겨울,0.000000,러블리,0.00
3,러바드 플레어 니트원피스-브라운,미니원피스,무지,플레어,겨울,4.842105,러블리,0.00
4,러바드 플레어 니트원피스-블랙,미니원피스,무지,플레어,겨울,4.842105,러블리,0.00
5,[La belle rose]돌체 자켓원피스(벨트set)-베이지,미니원피스,무지,단추,봄 가을,0.000000,로맨틱,0.00
6,[La belle rose]돌체 자켓원피스(벨트set)-블랙,미니원피스,무지,단추,봄 가을,0.000000,로맨틱,0.00
7,[La belle rose]원앤 슬림핏 자켓원피스-블랙,미니원피스,무지,포켓,겨울,0.000000,로맨틱,-0.04
8,[La belle rose]원앤 슬림핏 자켓원피스-블랙,미니원피스,무지,포켓,겨울,0.000000,로맨틱,-0.04
9,[La belle rose]원앤 슬림핏 자켓원피스-차콜,미니원피스,무지,포켓,겨울,0.000000,로맨틱,-0.04


In [271]:
get_recommendations('[La belle rose]론다 플리츠 미니스커트-베이지')

,상품이름,카테고리,패턴재질,모양,계절,평균평점,스타일,종합점수
0,[La belle rose]론다 플리츠 미니스커트-블랙,미니스커트,무지,플리츠,봄 가을,0.000000,러블리,0.0
1,[La belle rose]론다 플리츠 미니스커트-블랙,미니스커트,무지,플리츠,봄 가을,0.000000,러블리,0.0
2,민니 핀턱 주름스커트-블랙,미니스커트,무지,핀턱,봄 가을,4.607383,러블리,-0.1
3,민니 핀턱 주름스커트-아이보리,미니스커트,무지,핀턱,봄 가을,4.607383,러블리,-0.1
4,민니 핀턱 주름스커트-차콜,미니스커트,무지,핀턱,봄 가을,4.607383,러블리,-0.1
5,블라디 플리츠스커트-그레이,미니스커트,무지,플리츠,가을,4.785714,러블리,0.0
6,블라디 플리츠스커트-베이지,미니스커트,무지,플리츠,가을,4.785714,러블리,0.0
7,아세르 플리츠 미니스커트-라이트퍼플,미니스커트,무지,플리츠,봄 여름 가을,4.555556,러블리,0.0
8,아세르 플리츠 미니스커트-블랙,미니스커트,무지,플리츠,봄 여름 가을,4.555556,러블리,0.0
9,아세르 플리츠 미니스커트-크림,미니스커트,무지,플리츠,봄 여름 가을,4.555556,러블리,0.0
